<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/NLP/5_Atencion/ejercicios/ejercicios.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Usando atención.

Volveremos a usar las interfaces que usamos antes.

In [ ]:
import torch
import torch.nn as nn
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()

    # Más tarde puede haber argumentos adicionales
    # (por ejemplo, longitud para excluir el relleno)
    def forward(self, X, *args):
        raise NotImplementedError

In [ ]:
class Decoder(nn.Module):
    """The base decoder interface for the encoder-decoder architecture."""
    def __init__(self):
        super().__init__()

    # Más tarde puede haber argumentos adicionales
    # (por ejemplo, longitud para excluir el relleno)
    def init_state(self, enc_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        return self.decoder(dec_X, dec_state)[0]

# Ejercicio 1

Implementar un Encoder con GRU:

El constructor debe recibir los siguientes parametros:

`__init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim,
                 num_layers, dropout)`
  * `input_dim`: tamaño del vocabulario
  * `emb_dim`: tamaño del vector a la salida de una capa de embeddings
  * `enc_hid_dim`: tamaño del vector de estado oculto a la salida de GRU
  * `dec_hid_dim`: tamaño del vector de estado oculto a la salida de GRU **PERO EN EL DECODER**
  * `num_layers`: número de capas de GRU
  > NOTA: ese número de capas sera el mismo para el decoder y el encoder
  * `dropout`: probabilidad de dropout

La red debe hacer lo siguiente:
  1. Recibe un minilote de secuencias de tokens `src` y lo alimenta a una capa de `Embedding`
  1. Toma la salida de la capa `Embedding`y la usa en una `GRU` bidireccional, multicapa.
  2. Usar una capa densa `Linear`, para adaptar los estados ocultos del encoder al decoder. Esta capa recibe entra con `enc_hid_dim * num_layers * 2` y devuelve `dec_hid_dim * num_layers`
    * `GRU` bidireccional tiene como salida el último estado oculto de cada capa `hidden` y la secuencia temporal de los estado ocultos de la última capa `outputs`. 
    * `hidden` tiene  la forma `[n_capas * num_ direcciones, batch, hid_dim]`. Los estados ocultos estan ordenadados de la siguiente forma `[forward_1, backward_1, forward_2, backward_2, ...]`
    * deberá adaptar el tensor `hidden` para que tenga la forma `[1, batch, hid_dim * n_capas * num_ direcciones]` y luego alimentarlo a la capa lineal.
    * finalmente, deberá volver a adaptar la salida para que tenga la forma `[n_capas * num_ direcciones, batch, hid_dim]`.
    * Recuerde que el decoder es unidireccional

  1. La red debe devolver la salida de la capa densa y los `outputs` de GRU

In [ ]:
def init_RNN(module):
    if type(module) == nn.Linear:
         nn.init.xavier_uniform_(module.weight)
    if type(module) == nn.GRU:
        for param in module._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(module._parameters[param])

class ATTNEncoder(Encoder):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim,
                 num_layers, dropout):
        ### inserte su código
        
    def forward(self, src):
        
        ### inserte su código
        
        return outputs, hidden

Use esta  celda para verificar que todo funcione como corresponde

In [ ]:
vocab_size, embed_size, num_hiddens, num_layers = 10, 8, 16, 2
batch_size, num_steps = 4, 9

encoder = ATTNEncoder(vocab_size, embed_size, num_hiddens, num_hiddens, 
                      num_layers, 0.5)
X = torch.zeros((num_steps, batch_size), dtype=torch.int32)
outputs, state = encoder(X)
print(outputs.shape, (num_steps,batch_size,num_hiddens * 2), "todos los numeros  deben coincidir")
print(state.shape, (num_layers, batch_size,num_hiddens), "todos los numeros  deben coincidir")


torch.Size([9, 4, 32]) (9, 4, 32)
torch.Size([2, 4, 16]) (2, 4, 16)


# Ejercicio 2

Implementar un Decoder con GRU:

El constructor debe recibir los siguientes parametros:

`__init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, num_layers, dropout, attention):`
  * `output_dim`: tamaño del vector del vocabulario
  * `emb_dim`: tamaño del vector a la salida de una capa de embeddings
  * `enc_hid_dim`: tamaño del vector de estado oculto a la salida de GRU
  * `dec_hid_dim`: tamaño del vector de estado oculto a la salida de GRU **PERO EN EL DECODER**
  * `num_layers`: número de capas de GRU
  > NOTA: ese número de capas sera el mismo para el decoder y el encoder
  * `dropout`: probabilidad de dropout
  * `attention`: capa de atención.

La red debe hacer lo siguiente:
  1. Recibe un minilote de secuencias de tokens `input`, el estado oculto a la salida del encoder `hidden` y la secuencia de estados del decoder `encoder_outputs`
  1. Aplicamos un `unsqueeze` a `input` en la dimensión `0`
  1. Usamos `input`para alimentarlo a la alimenta a una capa de `Embedding`
  1. Alimentamos a la capa de atención `attention` con `hidden` y `encoder_outputs` para generar los pesos `a`.
  2. Generamos un estado con atención `weighted` a partir de los pesos `a` y `encoder_outputs`
    * `a` tiene la forma `[batch_size, sequence_len]` y `encoder_outputs` tiene la forma `[sequence_len, batch_size, end_hid_dim]` debemos transformarlos de la siguiente manera:
      * `a` --> `[batch_size, 1, sequence_len]`
      * `encoder_outputs` --> `[batch_size, sequence_len, end_hid_dim]`

      De esa manera podemos aplicar la función bmm de torch (batched matrix multiplication)
    * `weighted` tendra la forma `[batch_size, 1, sequence_len]`, pero necesitamos que tenga la forma `[1, batch_size, sequence_len]`
  3. Concatenamos `weighted` con la salida de la capa de embeddings en el tensor `input_rnn`
  4. Le damos a nuestra GRU, `input_rnn` y `hidden`. Recuerde que esta nos dara un nuevo `hidden` junto con un tensor `outputs`
  4. Tomamos la salida de nuestra GRU `outputs`, el tensor `weighted` y la salida del embedding y las concatenamos, eliminando la primera dimensión (`squeeze` en `0`)
  2. A la concatenación anterior, se la enviamos a una capa densa que nos predice el próximo token.
  3. La red devuelve las predicciones junto con el estado oculto `hidden recien calculado.`

In [ ]:
class ATTNDecoder(Decoder):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, num_layers, dropout, attention):
        super().__init__()

        ### inserte su código

        
    def forward(self, input, hidden, encoder_outputs):

        ### inserte su código
        
        return prediction, hidden

# Ejercicio 3

Implementar un Decoder con GRU:

El constructor debe recibir los siguientes parametros:

`    def __init__(self, enc_hid_dim, dec_hid_dim, num_layers):
`
  * `enc_hid_dim`: tamaño del vector de estado oculto a la salida de GRU
  * `dec_hid_dim`: tamaño del vector de estado oculto a la salida de GRU **PERO EN EL DECODER**
  * `num_layers`: número de capas de GRU
  > NOTA: ese número de capas sera el mismo para el decoder y el encoder

La red debe hacer lo siguiente:

  1. Recibe el estado oculto `hidden` y la salida del encoder `encoder_outputs`
  1. Convertimos el estado oculto `hidden` de un tensor de la forma `[num_layers, batch size, dec hid dim]` a uno de la forma `[batch size, 1, dec hid dim * num layers]`
  1. Repetimos en la dimensión 1 al tensor  `len_src` veces. Pasamos de un tensor de la forma `[batch size, 1, dec hid dim * num layers]` a uno de la forma `[batch size, src len, dec hid dim * num layers]`
  1. Convertimos el estado oculto `encoder_outputs` de un tensor de la forma `[src len, batch size, enc hid dim]` a uno de la forma `[batch size, src len, enc hid dim * num direction]`
  2. Aplicamos una capa densa seguida de una `tanh`
  3. Luego aplicamos una segunda capa densa. La salida debe tener la forma `[batch size, src len, 1]`. Por esta razón eliminaremos la última dimensión usando unsqueeze
  4. Devolvemos los pesos luego de aplicar un softmax en la dimensión `1`. La salida debe ser de la forma `[batch size, src len]`


In [ ]:
import torch.nn.functional as F
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim, num_layers):
        super().__init__()
        
        ### inserte su código
        
    def forward(self, hidden, encoder_outputs):
        
        ### inserte su código


        return F.softmax(attention, dim=1)

In [ ]:
import random
class ATTNSeq2Seq(EncoderDecoder):
    def __init__(self, encoder, decoder, device):
        super().__init__(encoder, decoder)
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        """
        Eliminamos el assert
        """
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        """
        Como ahora usamos anteción, el estado inicial es manejado
        por el decoder.
        """
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
# La siguiente celda no funcionará hasta que se cargue el vocabulario
# y se establezca el índice de padding
#loss = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

## Cargando los Datos



In [ ]:
import os.path
import re
from shutil import unpack_archive

data = None
!wget -O spa-eng.zip http://www.manythings.org/anki/spa-eng.zip
if not os.path.isfile("spa.txt"):
    unpack_archive('./spa-eng.zip', extract_dir='./', format='zip')
with open('./spa.txt', encoding='utf-8') as f:
    data = f.read()
    data = re.sub("\tCC-BY 2\.0.*","",data) # acá elimino información adicional
    data = re.sub(r"[\u202f]|[\xa0]"," ",data) # aca saco caracteres raros
    data = re.sub("([,\.:;!?])"," \\1",data) # aca  y abajo tokenizo puntuación
    data = re.sub("([¡¿])","\\1 ",data).lower()


--2022-08-30 12:55:00--  http://www.manythings.org/anki/spa-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5320329 (5.1M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   5.07M  9.40MB/s    in 0.5s    

2022-08-30 12:55:01 (9.40 MB/s) - ‘spa-eng.zip’ saved [5320329/5320329]



In [ ]:
print(data[:100])

go .	ve .
go .	vete .
go .	vaya .
go .	váyase .
hi .	hola .
run !	¡ corre !
run !	¡ corran !
run !	¡


In [ ]:
import random

SRC_IDX, TGT_IDX = 0, 1

SEED = 12312

data2 = data.split('\n')
random.seed(SEED)
random.shuffle(data2)

data_list = []
for i, line in enumerate(data2):
    parts = line.split('\t')
    if len(parts) == 2:
        # Skip empty tokens
        new_src = [t for t in f'{parts[SRC_IDX]} <eos>'.split(' ') if t]
        new_tgt = [t for t in f'<bos> {parts[TGT_IDX]} <eos>'.split(' ') if t]
        length_src = len(new_src)
        data_list.append((new_src, length_src, new_tgt))

print(data_list[0][0], data_list[0][-1])
print(len(data_list))

['tom', 'seldom', 'puts', 'sugar', 'in', 'his', 'coffee', '.', '<eos>'] ['<bos>', 'tom', 'casi', 'nunca', 'le', 'pone', 'azúcar', 'al', 'café', '.', '<eos>']
138440


In [ ]:
from collections import Counter
from torchtext.vocab import vocab

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n = len(data_list)
split1, split2 = int(0.7*n), int(0.9*n)
train_list = data_list[:split1]
val_list = data_list[split1:split2]
test_list = data_list[split2:]

counter_src, counter_tgt = Counter(), Counter()
for i in range(len(train_list)):
  counter_src.update(train_list[i][0])
  counter_tgt.update(train_list[i][-1])

vocab_src = vocab(counter_src, min_freq = 2,
              specials=('<unk>', '<eos>', '<bos>', '<pad>'))
vocab_src.set_default_index(vocab_src['<unk>'])

vocab_tgt = vocab(counter_tgt, min_freq = 2,
              specials=('<unk>', '<eos>', '<bos>', '<pad>'))
vocab_tgt.set_default_index(vocab_tgt['<unk>'])


In [ ]:
from torch.utils.data.sampler import Sampler

class BucketSampler(Sampler):

    def __init__(self, batch_size, train_list):
        self.length = len(train_list)
        self.train_list = train_list
        self.batch_size = batch_size
        indices = [(i, s[1]) for i, s in enumerate(self.train_list)]
        random.seed(SEED)
        random.shuffle(indices)
        pooled_indices = []
        # creamos minilotes de tamaños similares
        for i in range(0, len(indices), batch_size * 100):
            pooled_indices.extend(sorted(indices[i:i + batch_size * 100],
                                         key=lambda x: x[1], reverse=True))

        self.pooled_indices = pooled_indices

    def __iter__(self):
        for i in range(0, len(self.pooled_indices), self.batch_size):
            yield [idx for idx, _ in self.pooled_indices[i:i + self.batch_size]]

    def __len__(self):
        return (self.length + self.batch_size - 1) // self.batch_size

In [ ]:
import torch

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

SRC_PAD_IDX = vocab_src['<pad>']
TGT_PAD_IDX = vocab_tgt['<pad>']

def collate_batch(batch):
    text_src, length_list, text_tgt_in, text_tgt_out = [], [], [], []
    for (src, length, tgt) in batch:
        # convertimos el texto en tokens
        processed_src = torch.tensor([vocab_src[token] for token in src])
        processed_tgt = torch.tensor([vocab_tgt[token] for token in tgt])
        text_src.append(processed_src)
        text_tgt_in.append(processed_tgt[:-1])
        text_tgt_out.append(processed_tgt[1:])
        # guardamos la longitud de cada token
        length_list.append(length)
    # armamos la tupla que conformara un ejemplo de minilote.
    result = (pad_sequence(text_src, padding_value=SRC_PAD_IDX),
              pad_sequence(text_tgt_in, padding_value=TGT_PAD_IDX),
              pad_sequence(text_tgt_out, padding_value=TGT_PAD_IDX),)
    return result

In [ ]:
batch_size = 64  # A batch size of 64

train_bucket = BucketSampler(batch_size, train_list)
train_iter = DataLoader(train_list,
                          batch_sampler=train_bucket,
                          collate_fn=collate_batch)

val_bucket = BucketSampler(batch_size, val_list)
val_iter = DataLoader(val_list,
                          batch_sampler=val_bucket,
                          collate_fn=collate_batch)

test_bucket = BucketSampler(batch_size, test_list)
test_iter = DataLoader(test_list,
                          batch_sampler=test_bucket,
                          collate_fn=collate_batch)

In [ ]:
src, input, out = next(iter(test_iter))

a = [vocab_src.get_itos()[token] for example in src.T[:1] for token in example]
b = [vocab_tgt.get_itos()[token] for example in out.T[:1] for token in example]
c = [vocab_tgt.get_itos()[token] for example in input.T[:1] for token in example]

print(a)
print(b)
print(c)

['in', '1969', ',', 'roger', 'miller', '<unk>', 'a', 'song', 'called', '"you', "don't", 'want', 'my', 'love', '."', 'today', ',', 'this', 'song', 'is', 'better', 'known', 'as', '<unk>', 'the', 'summer', 'time', '."', "it's", 'the', 'first', 'song', 'he', 'wrote', 'and', 'sang', 'that', 'became', 'popular', '.', '<eos>']
['en', '1969', ',', 'roger', 'miller', 'grabó', 'una', 'canción', 'llamada', '<unk>', 'no', 'quieres', 'mi', '<unk>', '.', 'hoy', ',', 'esta', 'canción', 'es', 'más', 'conocida', 'como', '"en', 'el', '<unk>', '.', 'es', 'la', 'primera', 'canción', 'que', 'escribió', 'y', 'cantó', 'que', 'se', 'convirtió', 'popular', '.', '<eos>']
['<bos>', 'en', '1969', ',', 'roger', 'miller', 'grabó', 'una', 'canción', 'llamada', '<unk>', 'no', 'quieres', 'mi', '<unk>', '.', 'hoy', ',', 'esta', 'canción', 'es', 'más', 'conocida', 'como', '"en', 'el', '<unk>', '.', 'es', 'la', 'primera', 'canción', 'que', 'escribió', 'y', 'cantó', 'que', 'se', 'convirtió', 'popular', '.']


## Bucle de Entrenamiento

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src, tgt_input, tgt_out = batch
        src, tgt_input, tgt_out = src.to(device), tgt_input.to(device), tgt_out.to(device)        
        #src: son las frases en el idioma origen que le pasaremos como entrada al encoder
        #src.shape : [src len, batch size]
        #tgt_input: son las frases en el idioma destino que le pasaremos como entrada al decoder (con `<bos>` como primer token y sin `<eos>`)
        #tgt_out: son las frases en el idioma destino que usaremos para calcular la pérdida (con `<eos>` como finalizador de oración y sin `<bos>`)
        #tgt.shape : [trg len, batch size]
        
        optimizer.zero_grad()
        output = model(src, tgt_input)
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        #como la función de pérdida solo funciona en entradas 2d con objetivos 1d,
        # necesitamos aplanar cada una de ellas con .view
        output = output[1:].view(-1, output_dim)
        trg = tgt_out[1:].view(-1)
        #trg = [trg len * batch size]
        #output = [trg len * batch size, output dim]
        
        #calculamos los gradientes
        loss = criterion(output, trg)
        loss.backward()

        #recortamos los gradientes para evitar que exploten
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0 
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, tgt_input, tgt_out = batch
            src, tgt_input, tgt_out = src.to(device), tgt_input.to(device), tgt_out.to(device)    
            output = model(src, tgt_input, 0) #turn off teacher forcing
            #output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]

            output = output.view(-1, output_dim)
            trg = tgt_out.view(-1)
            #trg = [trg len * batch size]
            #output = [trg len * batch size, output dim]

            loss = criterion(output, trg)            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Entrenamiento.

Luego, entrenamos nuestro modelo, guardando los parámetros que nos den la mejor pérdida de validación.

In [ ]:
import torch.optim as optim

INPUT_DIM = len(vocab_src.get_itos())
OUTPUT_DIM = len(vocab_tgt.get_itos())
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, N_LAYERS)
enc = ATTNEncoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = ATTNDecoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT, attn)

model = ATTNSeq2Seq(enc, dec, device).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = TGT_PAD_IDX)

In [ ]:
import math
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 3m 42s
	Train Loss: 4.055 | Train PPL:  57.687
	 Val. Loss: 4.257 |  Val. PPL:  70.598
Epoch: 02 | Time: 3m 43s
	Train Loss: 2.919 | Train PPL:  18.526
	 Val. Loss: 4.255 |  Val. PPL:  70.481
Epoch: 03 | Time: 3m 43s
	Train Loss: 2.580 | Train PPL:  13.204
	 Val. Loss: 4.088 |  Val. PPL:  59.617
Epoch: 04 | Time: 3m 43s
	Train Loss: 2.420 | Train PPL:  11.244
	 Val. Loss: 4.144 |  Val. PPL:  63.054
Epoch: 05 | Time: 3m 43s
	Train Loss: 2.341 | Train PPL:  10.396
	 Val. Loss: 4.160 |  Val. PPL:  64.067
Epoch: 06 | Time: 3m 43s
	Train Loss: 2.296 | Train PPL:   9.935
	 Val. Loss: 4.173 |  Val. PPL:  64.898
Epoch: 07 | Time: 3m 43s
	Train Loss: 2.264 | Train PPL:   9.621
	 Val. Loss: 4.165 |  Val. PPL:  64.383
Epoch: 08 | Time: 3m 44s
	Train Loss: 2.248 | Train PPL:   9.466
	 Val. Loss: 4.191 |  Val. PPL:  66.114
Epoch: 09 | Time: 3m 43s
	Train Loss: 2.234 | Train PPL:   9.341
	 Val. Loss: 4.226 |  Val. PPL:  68.414
Epoch: 10 | Time: 3m 42s
	Train Loss: 2.224 | Train PPL

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.109 | Test PPL:  60.867 |


## Predicción

In [ ]:
src = "i'm cold . <eos>"
tgt = "tengo frío ."

### Solo debe generar predicciones, no hace falta que devuelva los pesos.
def prediction(src, tgt, max_len): 
  tkn_src = [ vocab_src[tkn] for tkn in src.split(' ')]
  input_src = torch.Tensor(tkn_src)
  input_src = input_src.type(torch.int32)
  input_src = input_src.unsqueeze(1)
  input_src = input_src.to(device)

  model.eval()
  outputs, hidden = model.encoder(input_src)

  final = []
  input_tgt = torch.Tensor([vocab_tgt['<bos>']])
  for i in range(max_len):
      input_tgt = input_tgt.type(torch.int32)
      input_tgt = input_tgt.to(device)
      next_tkn, hidden = model.decoder(input_tgt, hidden, outputs)
      input_tgt = next_tkn.argmax(1)
      string = vocab_tgt.get_itos()[int(input_tgt)]
      if string == '<eos>':
          break
      final.append(string)
  return final

final = prediction(src, tgt, 9)
print(src)
print(tgt)
print(final)

i'm cold . <eos>
tengo frío .
['frío', '.']


In [ ]:
import collections

def bleu(pred_seq, label_seq, k):
    # convertimos nuestras oraciones en secuencias de palabras
    pred_tokens, label_tokens = pred_seq.split(' '), label_seq.split(' ')

    # calculamos la longitud
    len_pred, len_label = len(pred_tokens), len(label_tokens)

    # iniciamos el primer valor de puntaje.
    score = math.exp(min(0, 1 - len_label / len_pred))

    # ciclo
    for n in range(1, min(k, len_pred) + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)

        for i in range(len_label - n + 1):
            ## Generamos todos los n-gramas de tamaño i para labels
            label_subs[' '.join(label_tokens[i: i + n])] += 1

        for i in range(len_pred - n + 1):
            ## Generamos todos los n-gramas de tamaño i para labels
            if label_subs[' '.join(pred_tokens[i: i + n])] > 0:
                #contamos coincidencias.
                num_matches += 1
                label_subs[' '.join(pred_tokens[i: i + n])] -= 1

        # actualizamos score
        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    return score

In [ ]:
print(bleu(' '.join(final),tgt,2))

0.6065306597126334
